In [4]:
!pip install sentence-transformers transformers faiss-cpu PyPDF2 langchain


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 17.8 MB/s eta 0:00:00


In [5]:
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import faiss
import numpy as np

# Load models
embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
generator = pipeline("text2text-generation", model="google/flan-t5-base")

# Example documents
docs = [
    "Python is a popular programming language used for AI and data science.",
    "Machine learning allows computers to learn from data.",
    "RAG stands for Retrieval-Augmented Generation."
]

# Step 1: Create embeddings
doc_embeddings = embedder.encode(docs, convert_to_tensor=False)
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(np.array(doc_embeddings))

# Step 2: Query
query = "What is RAG in AI?"
query_embedding = embedder.encode([query])
D, I = index.search(np.array(query_embedding), k=2)

# Step 3: Retrieve relevant docs
retrieved_docs = [docs[i] for i in I[0]]

# Step 4: Generate answer
context = " ".join(retrieved_docs)
prompt = f"Using the context below, answer the question.\nContext: {context}\nQuestion: {query}"
answer = generator(prompt, max_length=100)[0]['generated_text']

print("Answer:", answer)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer: Retrieval-Augmented Generation


In [7]:
docs = [
    """Python is a high-level, versatile programming language known for its simplicity and readability.
    It was created by Guido van Rossum and first released in 1991. Here's what makes Python special:

    🐍 Key Features of Python
    Readable Syntax: Python’s syntax is clean and close to English, making it beginner-friendly.
    Interpreted Language: You don’t need to compile Python code—just run it directly.
    Object-Oriented: Supports classes, inheritance, and other OOP principles.
    Dynamic Typing: You don’t need to declare variable types explicitly.
    Extensive Libraries: From web development to data science, Python has libraries for almost everything.

    🛠️ What Can You Build with Python?
    Web applications using frameworks like Django or Flask.
    Data analysis and machine learning with libraries like Pandas, NumPy, and TensorFlow.
    Automation scripts for tasks like file handling or web scraping.
    Games, GUIs, and even IoT applications.

    💡 Why Developers Love Python
    Cross-platform compatibility (Windows, Mac, Linux).
    Huge community and tons of learning resources.
    Great for rapid prototyping and scalable production systems.
    """
]

# ✅ STEP 4: Load embedding model & LLM
embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
generator = pipeline("text2text-generation", model="google/flan-t5-base")

# ✅ STEP 5: Convert documents into embeddings
doc_embeddings = embedder.encode(docs, convert_to_tensor=False)
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(np.array(doc_embeddings))

# ✅ STEP 6: Function to answer user query
def answer_question(query):
    query_embedding = embedder.encode([query])
    D, I = index.search(np.array(query_embedding), k=1)
    retrieved_doc = docs[I[0][0]]
    prompt = f"Using the following context, answer the question briefly.\n\nContext: {retrieved_doc}\n\nQuestion: {query}"
    result = generator(prompt, max_length=150)[0]['generated_text']
    return result

# ✅ STEP 7: Try a query
query = "What are the key features of Python?"
print("Question:", query)
print("Answer:", answer_question(query))


Device set to use cpu


Question: What are the key features of Python?


Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer: Readable Syntax: Python’s syntax is clean and close to English, making it beginner-friendly. Interpreted Language: You don’t need to compile Python code—just run it directly. Object-Oriented: Supports classes, inheritance, and other OOP principles. Dynamic Typing: You don’t need to declare variable types explicitly. Extensive Libraries: From web development to data science, Python has libraries for almost everything.  What Can You Build with Python? Web applications using frameworks like Django or Flask. Data analysis and machine learning with libraries like Pandas, NumPy, and TensorFlow. Automation scripts for tasks like file handling or web scraping. Games, GUIs, and even IoT applications.  Why Developers Love Python Cross-platform compatibility (Windows, Mac, Linux). Huge community and tons of learning resources. Great for rapid prototyping and scalable production systems.


In [10]:

import gradio as gr

def rag_search(query):
    query_embedding = embedder.encode([query])
    D, I = index.search(np.array(query_embedding), k=1)
    retrieved_doc = docs[I[0][0]]
    prompt = f"Using this context, answer the question briefly.\n\nContext: {retrieved_doc}\n\nQuestion: {query}"
    result = generator(prompt, max_length=350)[0]['generated_text']
    return result

gr.Interface(
    fn=rag_search,
    inputs="text",
    outputs="text",
    title="Knowledge-base Search Engine",
    description="Ask questions about the uploaded knowledge base (Python, ML, etc.)"
).launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e375ec91bb1bac4c57.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
